# INIT

START

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config

import ipytest
ipytest.autoconfig()
from time import sleep

from context import Context
from utils.utils_chain import WrapperAddress as Address, Account, hex_to_string, base64_to_hex
from utils.utils_chain import nominated_amount, get_all_token_nonces_details_for_account
from utils.utils_tx import endpoint_call

from contracts.farm_contract import FarmContract
from utils.contract_retrievers import retrieve_farm_by_address

from multiversx_sdk import ProxyNetworkProvider, ApiNetworkProvider

from pprint import pprint

SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"
GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL = f"{SIMULATOR_URL}/simulator/generate-blocks-until-epoch-reached"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api

In [ ]:
farm_contract: FarmContract = context.deploy_structure.get_deployed_contract_by_index(config.FARMS_V2, 0)
print(f"Using {farm_contract.address} : {farm_contract.farmToken}")

In [ ]:
from contracts.unstaker_contract import UnstakerContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract

simple_lock_energy_contract: SimpleLockEnergyContract
simple_lock_energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
print(f"simple_lock_energy_contract: {simple_lock_energy_contract.address}")

unstaker_contract: UnstakerContract
unstaker_contract = context.get_contracts(config.UNSTAKERS)[0]
print(f"unstaker_contract: {unstaker_contract.address}")

CHAIN CONFIG SETUP

In [4]:
import json
from typing import Any
from utils.utils_chain import WrapperAddress

USERS = ["erd1ndyxz4gpfncmz82qm39yqramgd826mkalhqw8tfmmy0g9jecqc5sa20vek", # user with both farm and farming positions
         "erd1gqyspww4pssv6ck6pl8vtnl9tnwe9hy5d2324mya4rz5ma9dtp9snjgd7f", # user with only farm position
         "erd15gzp9k56cnn8qtfxwlghcxgs74v8jmfk4crex9alygxldmpg9f5s3fd4pl", # user with old farm position
         "erd1yhuhzm8uu4efdts924e50wvaquhx8xg2c038sdu7n8uyh3cgxqeqww97m9", # user with lots of xmex
         "erd1qqqqqqqqqqqqqqqpqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqzllls8a5w6u",
         # "erd1lllllllllllllllllllllllllllllllllllllllllllllllllllsckry7t"
]

def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    

def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post(f"{SIMULATOR_URL}/simulator/set-state", state)


def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def setup_chain_sim():
    # generate blocks to pass an epoch and the smart contract deploys to be enabled
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/5", {})

    load_and_apply_state(proxy, PROJECT_ROOT,
                         context.deployer_account.address.bech32(),
                         USERS)


def advance_blocks(number_of_blocks: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})

def advance_to_epoch(epoch: int):
    proxy.do_post(f"{GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL}/{epoch}", {})


def users_init() -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in USERS:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

Farm upgrade

In [5]:
def farm_upgrade():
    wasm_path = config.HOME / "projects/dex/dex-v2/sc-dex-rs/builds/farm-with-locked-rewards.wasm"
    tx_hash = farm_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                            wasm_path, 
                                            [], True)

    advance_blocks(1)
    tx_hash = farm_contract.resume(context.deployer_account, context.network_provider.proxy)
    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(farm_contract.address)).code_hash
    assert base64_to_hex(code_hash) == "a59bdb9c06610a167c1344afc23d6a2d3516892f3a5aca165fb4a0f924e9d6d9"

Unstaker upgrade

In [6]:
def unstaker_upgrade():
    wasm_path = config.HOME / "projects/dex/dex-v2/sc-dex-rs/output-docker/token-unstake/token-unstake.wasm"
    tx_hash = unstaker_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, 
                                            wasm_path, 
                                            [], True)

    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(unstaker_contract.address)).code_hash
    assert base64_to_hex(code_hash) == "2abf0e8b612ac9e58f54fe82da289e9c5a498863386bbeac0518a216eb043b42"

Deploy permissions hub

In [7]:
from contracts.permissions_hub_contract import PermissionsHubContract

def deploy_permissions_hub():
    permissions_hub_contract = PermissionsHubContract("")
    _, address = permissions_hub_contract.contract_deploy(context.deployer_account, context.network_provider.proxy, 
                                            "https://github.com/multiversx/mx-exchange-sc/releases/download/v3.2.2-rc2/permissions-hub.wasm",
                                            [])
    permissions_hub_contract.address = address
    return permissions_hub_contract

Dummy proxy

In [8]:
from contracts.dummy_proxy_contract import DummyProxyContract

def deploy_dummy_proxy_contract():
    dummy_proxy_contract = DummyProxyContract("")
    _, address = dummy_proxy_contract.contract_deploy(context.deployer_account, context.network_provider.proxy,
                                         "https://github.com/ovidiuolteanu/mx-sc-dummy-proxy/releases/download/v2.1/dummy-proxy.wasm",
                                         [])
    dummy_proxy_contract.address = address
    return dummy_proxy_contract

In [9]:
from utils.utils_chain import get_all_token_nonces_details_for_account

def get_position_for_account(user_address: str):
    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")
    
    return farm_tk_nonce, farm_tk_balance

def get_farming_token_balance_for_account(user_address: str):
    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_address, context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmingToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")
    
    return farming_tk_balance

In [10]:
from utils.utils_tx import ESDTToken, multi_esdt_transfer

def send_tokens_to_dummy(user: Account, dummy_contract: DummyProxyContract):
    farm_tk_nonce, farm_tk_balance = get_position_for_account(user)
    print(f"Sending {farm_tk_balance} {farm_contract.farmToken}-{farm_tk_nonce} to dummy contract")
    multi_esdt_transfer(context.network_provider.proxy, 20000000, user, dummy_contract.address, [ESDTToken(farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)])

Claim

In [11]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

    tx_hash = farm_contract.claimRewards(context.network_provider, user_account, event)
    return tx_hash

Claim boosted

In [12]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_boosted_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)
    event = ClaimRewardsFarmEvent(0, 0, '', user_account.address.bech32())
    tx_hash = farm_contract.claim_boosted_rewards(context.network_provider, user_account, event)
    return tx_hash

Claim on behalf

In [13]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import ClaimRewardsFarmEvent

def claim_on_behalf_from_user(claim_account: Account):
    claim_account.sync_nonce(context.network_provider.proxy)

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, claim_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = ClaimRewardsFarmEvent(farm_tk_balance, farm_tk_nonce, '')

    tx_hash = farm_contract.claim_rewards_on_behalf(context.network_provider, claim_account, event)
    return tx_hash

Enter farm consolidated

In [14]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance)

    tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event)
    return tx_hash

Enter farm no consolidation

In [15]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_no_consolidation_for_user(user_account: Account):
    user_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                       "", 0, 0)

    tx_hash = farm_contract.enterFarm(context.network_provider, user_account, event, 0, True)
    return tx_hash

Enter farm on behalf

In [16]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_on_behalf_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    farm_tk_balance, farm_tk_nonce = 0, 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} positions of {farm_contract.farmToken} in account')
    for token in tokens_in_account:
        if int(token['balance']) > farm_tk_balance:
            farm_tk_balance = int(token['balance'])
            farm_tk_nonce = token['nonce']
            break

    if not farm_tk_nonce:
        raise Exception("Not enough farm token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                       farm_contract.farmToken, farm_tk_nonce, farm_tk_balance, user_account.address.bech32())

    tx_hash = farm_contract.enter_farm_on_behalf(context.network_provider, caller_account, event)
    return tx_hash

Enter farm on behalf no consolidation

In [17]:
from utils.utils_chain import Account, WrapperAddress as Address, get_all_token_nonces_details_for_account
from contracts.farm_contract import EnterFarmEvent

def enter_farm_on_behalf_no_consolidation_for_user(caller_account: Account, user_account: Account):
    caller_account.sync_nonce(context.network_provider.proxy)

    farming_tk_balance = 0
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, caller_account.address.bech32(), context.network_provider.proxy)
    print(f'Found {len(tokens_in_account)} farming tokens in account')
    for token in tokens_in_account:
        if int(token['balance']) > farming_tk_balance:
            farming_tk_balance = int(token['balance'])
            break

    if not farming_tk_balance:
        raise Exception("Not enough farming token balance")

    event = EnterFarmEvent(farm_contract.farmingToken, 0, farming_tk_balance,
                           "", 0, 0, user_account.address.bech32())

    tx_hash = farm_contract.enter_farm_on_behalf(context.network_provider, caller_account, event)
    return tx_hash

In [18]:
def dict_compare(d1, d2):
    print(d1)
    print(d2)
    d1_keys = set(d1.keys())
    d2_keys = set(d2.keys())
    shared_keys = d1_keys.intersection(d2_keys)
    added = d1_keys - d2_keys
    removed = d2_keys - d1_keys
    modified = {o : (d1[o], d2[o]) for o in shared_keys if d1[o] != d2[o]}
    same = set(o for o in shared_keys if d1[o] == d2[o])
    return added, removed, modified, same

def check_equal_dicts(dict1, dict2):
    """
    Compare two dictionaries, including nested dictionaries.
    
    Args:
    dict1 (dict): First dictionary to compare.
    dict2 (dict): Second dictionary to compare.
    
    Returns:
    bool: True if dictionaries are equal, False otherwise.
    """
    if dict1.keys() != dict2.keys():
        return False
    
    for key in dict1:
        if isinstance(dict1[key], dict) and isinstance(dict2[key], dict):
            if not check_equal_dicts(dict1[key], dict2[key]):
                return False
        elif dict1[key] != dict2[key]:
            return False
    
    return True

In [19]:
from utils.decoding_structures import FARM_TOKEN_ATTRIBUTES
from utils.utils_chain import decode_merged_attributes, base64_to_hex, WrapperAddress, get_all_token_nonces_details_for_account

def user_farm_token_stats(user):
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmingToken, user.address.bech32(), context.network_provider.proxy)
    print(f'Account: {user.address.bech32()}')
    print(f'Looking for {farm_contract.farmingToken} and {farm_contract.farmToken} tokens')
    print(f'Farming Tokens in account:')
    for token in tokens_in_account:
        print(f'\t{token}')
    tokens_in_account = get_all_token_nonces_details_for_account(farm_contract.farmToken, user.address.bech32(), context.network_provider.proxy)
    print(f'Farm Tokens in account:')
    all_decoded_attributes = []
    for token in tokens_in_account:
        print(f'\t{token}')
        decoded_attributes = decode_merged_attributes(base64_to_hex(token["attributes"]), FARM_TOKEN_ATTRIBUTES)
        print(f'\t\t{decoded_attributes}')
        all_decoded_attributes.append(decoded_attributes)
        
    return all_decoded_attributes

CHAIN SIMULATOR STACK

In [20]:
import subprocess
from time import sleep

CS_DOCKER_PATH = Path.home() / "projects/testing/full-stack-docker-compose/chain-simulator"

def start_chain_sim_stack():
    # stop first in case one is already running
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    
    p = subprocess.Popen(["docker", "compose", "up", "-d"], cwd = CS_DOCKER_PATH)
    sleep(60)
    return p

def stop_chain_sim_stack(p):
    p.terminate()
    p = subprocess.Popen(["docker", "compose", "down"], cwd = CS_DOCKER_PATH)
    p.wait()
    _ = subprocess.run(["docker", "system", "prune", "-f"], cwd = CS_DOCKER_PATH)

# SCENARIOS

Chain sim control

In [ ]:
chain_sim_stack = start_chain_sim_stack()

In [ ]:
stop_chain_sim_stack(chain_sim_stack)

Init

In [ ]:
setup_chain_sim()
users = users_init()
advance_blocks(1)

Unstaker

In [ ]:
from pprint import pprint
from utils.utils_chain import get_all_token_nonces_details_for_account

user = users[-2]
tokens = get_all_token_nonces_details_for_account(simple_lock_energy_contract.locked_token, user.address.bech32(), context.network_provider.proxy)
print(f"user: {user.address.bech32()} {simple_lock_energy_contract.locked_token} tokens: {len(tokens)}")

Unlock all tokens

In [ ]:

tokens = get_all_token_nonces_details_for_account(simple_lock_energy_contract.locked_token, user.address.bech32(), context.network_provider.proxy)

for token in tokens:
    print(f"Processing token: {token}")
    token_to_unlock = [ESDTToken(simple_lock_energy_contract.locked_token, int(token['nonce']), int(token['balance']))]
    tx_hash = simple_lock_energy_contract.unlock_early(user, context.network_provider.proxy, [token_to_unlock])
    print(tx_hash)
    advance_blocks(1)
advance_blocks(5)

In [ ]:
from utils.utils_tx import ESDTToken

user = users[-1]
tokens = get_all_token_nonces_details_for_account(simple_lock_energy_contract.locked_token, user.address.bech32(), context.network_provider.proxy)
print(f"user: {user.address.bech32()} {simple_lock_energy_contract.locked_token} tokens: {len(tokens)}")

for token in tokens:
    print(f"Processing token: {token}")
    token_to_unlock = [ESDTToken(simple_lock_energy_contract.locked_token, int(token['nonce']), int(token['balance']))]
    tx_hash = simple_lock_energy_contract.unlock_early(user, context.network_provider.proxy, [token_to_unlock])
    print(tx_hash)

In [ ]:
epochs_to_advance = 10
current_epoch = context.network_provider.proxy.get_network_status().epoch_number
print(f"Advancing from epoch {current_epoch} to {current_epoch + epochs_to_advance}")
advance_to_epoch(current_epoch + epochs_to_advance)
print(f"Done! Epoch after advancement: {context.network_provider.proxy.get_network_status().epoch_number}")

In [ ]:
from utils.utils_tx import endpoint_call
tx_hash = endpoint_call(proxy, 500000000, user, Address(unstaker_contract.address), "claimUnlockedTokens", [])
advance_blocks(10)

In [ ]:
print(context.get_contracts(config.FEES_COLLECTORS)[0].address)

In [22]:
with open(PROJECT_ROOT / "states" / "0_system_account_state_erd1lllllllllllllllllllllllllllllllllllllllllllllllllllsckry7t.json", "r") as file:
    retrieved_state = json.load(file)
    apply_states(proxy, [retrieved_state])

In [ ]:
unstaker_upgrade()
advance_blocks(1)

Unlock & unbond scenario

In [ ]:
user_index = 0
chain_sim_stack = start_chain_sim_stack()
setup_chain_sim()
users = users_init()
advance_blocks(1)

# get user
user = users[user_index]
tokens = get_all_token_nonces_details_for_account(simple_lock_energy_contract.locked_token, user.address.bech32(), context.network_provider.proxy)
print(f"user: {user.address.bech32()} {simple_lock_energy_contract.locked_token} tokens: {len(tokens)}")

# unlock all locked tokens
for token in tokens:
    print(f"Processing token: {token}")
    token_to_unlock = [ESDTToken(simple_lock_energy_contract.locked_token, int(token['nonce']), int(token['balance']))]
    tx_hash = simple_lock_energy_contract.unlock_early(user, context.network_provider.proxy, [token_to_unlock])
    print(tx_hash)
    advance_blocks(1)
advance_blocks(5)

# pass unbonding epochs
epochs_to_advance = 10
current_epoch = context.network_provider.proxy.get_network_status().epoch_number
print(f"Advancing from epoch {current_epoch} to {current_epoch + epochs_to_advance}")
advance_to_epoch(current_epoch + epochs_to_advance)
print(f"Done! Epoch after advancement: {context.network_provider.proxy.get_network_status().epoch_number}")

# TODO: temporary patch for unknown reasons
with open(PROJECT_ROOT / "states" / "0_system_account_state_erd1lllllllllllllllllllllllllllllllllllllllllllllllllllsckry7t.json", "r") as file:
    retrieved_state = json.load(file)
    apply_states(proxy, [retrieved_state])

unstaker_upgrade()
advance_blocks(1)

# unbond tokens
tx_hash = endpoint_call(proxy, 500000000, user, Address(unstaker_contract.address), "claimUnlockedTokens", [])
advance_blocks(10)

Burn 0%

In [ ]:
user_index = 0
chain_sim_stack = start_chain_sim_stack()
setup_chain_sim()
users = users_init()
advance_blocks(1)

# get user
user = users[user_index]
tokens = get_all_token_nonces_details_for_account(simple_lock_energy_contract.locked_token, user.address.bech32(), context.network_provider.proxy)
print(f"user: {user.address.bech32()} {simple_lock_energy_contract.locked_token} tokens: {len(tokens)}")

# unlock all locked tokens
for token in tokens:
    print(f"Processing token: {token}")
    token_to_unlock = [ESDTToken(simple_lock_energy_contract.locked_token, int(token['nonce']), int(token['balance']))]
    tx_hash = simple_lock_energy_contract.unlock_early(user, context.network_provider.proxy, [token_to_unlock])
    print(tx_hash)
    advance_blocks(1)
advance_blocks(5)

# pass unbonding epochs
epochs_to_advance = 10
current_epoch = context.network_provider.proxy.get_network_status().epoch_number
print(f"Advancing from epoch {current_epoch} to {current_epoch + epochs_to_advance}")
advance_to_epoch(current_epoch + epochs_to_advance)
print(f"Done! Epoch after advancement: {context.network_provider.proxy.get_network_status().epoch_number}")

# TODO: temporary patch for unknown reasons
with open(PROJECT_ROOT / "states" / "0_system_account_state_erd1lllllllllllllllllllllllllllllllllllllllllllllllllllsckry7t.json", "r") as file:
    retrieved_state = json.load(file)
    apply_states(proxy, [retrieved_state])

unstaker_upgrade()
advance_blocks(5)

# unbond tokens
tx_hash = endpoint_call(proxy, 500000000, user, Address(unstaker_contract.address), "claimUnlockedTokens", [])
advance_blocks(10)

# ------------------------------------------------------------------------------------------------
input("Press Enter to continue...")
stop_chain_sim_stack(chain_sim_stack)
chain_sim_stack = start_chain_sim_stack()
setup_chain_sim()
users = users_init()
advance_blocks(1)

# get user
user = users[user_index]
tokens = get_all_token_nonces_details_for_account(simple_lock_energy_contract.locked_token, user.address.bech32(), context.network_provider.proxy)
print(f"user: {user.address.bech32()} {simple_lock_energy_contract.locked_token} tokens: {len(tokens)}")

# unlock all locked tokens
for token in tokens:
    print(f"Processing token: {token}")
    token_to_unlock = [ESDTToken(simple_lock_energy_contract.locked_token, int(token['nonce']), int(token['balance']))]
    tx_hash = simple_lock_energy_contract.unlock_early(user, context.network_provider.proxy, [token_to_unlock])
    print(tx_hash)
    advance_blocks(1)
advance_blocks(5)

# pass unbonding epochs
epochs_to_advance = 10
current_epoch = context.network_provider.proxy.get_network_status().epoch_number
print(f"Advancing from epoch {current_epoch} to {current_epoch + epochs_to_advance}")
advance_to_epoch(current_epoch + epochs_to_advance)
print(f"Done! Epoch after advancement: {context.network_provider.proxy.get_network_status().epoch_number}")

# TODO: temporary patch for unknown reasons
with open(PROJECT_ROOT / "states" / "0_system_account_state_erd1lllllllllllllllllllllllllllllllllllllllllllllllllllsckry7t.json", "r") as file:
    retrieved_state = json.load(file)
    apply_states(proxy, [retrieved_state])

unstaker_upgrade()
advance_blocks(1)

unstaker_contract.set_fees_burn_percentage(context.deployer_account, context.network_provider.proxy, [0])
advance_blocks(4)

# unbond tokens
tx_hash = endpoint_call(proxy, 500000000, user, Address(unstaker_contract.address), "claimUnlockedTokens", [])
advance_blocks(10)